In [55]:
from keras.models import load_model
from keras.preprocessing import sequence
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [56]:
model = load_model("generated_data/complexity_classifier.h5")

In [57]:
easy = "hello my name is kevin and this is everything you need to know about me"

In [58]:
hard = "Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables (entities each of which takes on various numerical values) into a set of values of linearly uncorrelated variables called principal components."

In [59]:
import pickle
with open('generated_data/word_to_num.pkl', 'rb') as f:
    reverse_dict = pickle.load(f)

In [60]:
def get_sentence_complexity(sentence):
    num_list = []
    for word in sentence.split(" "):
        word = word.lower()
        remove_re = r'[!@#$%^&*()<>"{}|,+0-9~;`]'
        word = re.sub(remove_re, '', word)
        word = lemmatizer.lemmatize(word)
        if word in reverse_dict:
            num_list.append(reverse_dict[word])
    sentence = np.array(num_list)
    max_wiki_length = 100
    sentence = sequence.pad_sequences(np.array([sentence]), maxlen=max_wiki_length)
    result = model.predict(sentence)[0][0]
    if result < 0.5:
        return result, "simple"
    else:
        return result, "complex"

In [61]:
get_sentence_complexity('PCA of a multivariate Gaussian distribution centered at (1,3) with a standard deviation of 3 in roughly the (0.866, 0.5) direction and of 1 in the orthogonal direction. The vectors shown are the eigenvectors of the covariance matrix scaled by the square root of the corresponding eigenvalue, and shifted so their tails are at the mean. Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables (entities each of which takes on various numerical values) into a set of values of linearly uncorrelated variables called principal components. ')

(0.5043744, 'complex')

In [51]:
get_sentence_complexity(hard)

(0.5114905, 'complex')

In [52]:
get_sentence_complexity("this is supposed to be an easy to understand sentence")

(0.49973223, 'simple')

In [53]:
get_sentence_complexity("PCA can be done by eigenvalue decomposition of a data covariance (or correlation) matrix or singular value decomposition of a data matrix, usually after a normalization step of the initial data")

(0.5041232, 'complex')

In [54]:
get_sentence_complexity("pca is mostly used as a tool in exploratory data analysis and for making predictive models. It is often used to visualize genetic distance and relatedness between populations.")

(0.49936002, 'simple')